<a href="https://colab.research.google.com/github/Jeditem/VKCup_2020/blob/master/VKCup_ads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import pandas_profiling as pd_prof
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Анализ USERS

In [0]:
users_df = pd.read_csv(r'/content/gdrive/My Drive/VKCup/users.tsv', sep='\t')

In [0]:
users_df.describe()

,user_id,sex,age,city_id
count,27769.000000,27769.000000,27769.000000,27769.000000
mean,13884.000000,1.475134,28.894847,240.141165
std,8016.364149,0.501549,18.510084,466.122195
min,0.000000,0.000000,0.000000,0.000000
25%,6942.000000,1.000000,18.000000,0.000000
50%,13884.000000,1.000000,27.000000,25.000000
75%,20826.000000,2.000000,36.000000,231.000000
max,27768.000000,2.000000,118.000000,2456.000000


age has 2180 / 7.9% zeros Zeros
city_id has 8479 / 30.5% zeros Zeros

видимо в анкете здесь ничего ен указано, придется выкинуть

по user_id ничего интересного

в sex - тоже есть нули - значит анкета порченая
минимальный возраст - 13 лет, максимальный - 118 (аж 114 записей) - попахивает
ну, по городам - очевидно выделяется Москва, про все остальное рано говорить

## Профиль USERS

In [0]:
pd_prof.ProfileReport(users_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,4
Number of observations,27769
Total Missing (%),0.0%
Total size in memory,867.9 KiB
Average record size in memory,32.0 B
Numeric,4
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# Анализ HISTORY

In [0]:
history_df = pd.read_csv(r'/content/gdrive/My Drive/VKCup/history.tsv', sep='\t')

In [0]:
history_df.describe()

,hour,cpm,publisher,user_id
count,1.147857e+06,1.147857e+06,1.147857e+06,1.147857e+06
mean,7.603016e+02,1.864043e+02,2.004760e+00,1.384739e+04
std,4.306159e+02,4.557257e+02,2.061949e+00,8.028620e+03
min,3.000000e+00,3.000000e+01,1.000000e+00,0.000000e+00
25%,3.920000e+02,5.500000e+01,1.000000e+00,6.817000e+03
50%,7.620000e+02,1.100000e+02,1.000000e+00,1.394100e+04
75%,1.138000e+03,2.100000e+02,2.000000e+00,2.075900e+04
max,1.490000e+03,2.090540e+05,2.100000e+01,2.776800e+04


## Профиль HISTORY

In [0]:
pd_prof.ProfileReport(history_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,4
Number of observations,1147857
Total Missing (%),0.0%
Total size in memory,35.0 MiB
Average record size in memory,32.0 B
Numeric,4
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [0]:
validate_df = pd.read_csv(r'/content/gdrive/My Drive/VKCup/validate.tsv', sep='\t')

In [0]:
validate_df.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


In [0]:
answers_df = pd.read_csv(r'/content/gdrive/My Drive/VKCup/validate_answers.tsv', sep='\t')

In [0]:
answers_df.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


# Baseline

In [0]:
base_model =  LinearRegression()

## Data preparation

In [0]:
X_train_ini = validate_df.drop(columns=['user_ids'])
X_train_ini['ads_duration'] = X_train_ini['hour_end'] - X_train_ini['hour_start']
X_train_ini['publishers_amount'] = X_train_ini['publishers'].astype(str).str.split(',').str.len()
X_train_ini = X_train_ini.drop(columns=['publishers'])

In [0]:
X_train_ini.head()

,cpm,hour_start,hour_end,audience_size,ads_duration,publishers_amount
0,220.0,1058,1153,1906,95,2
1,312.0,1295,1301,1380,6,2
2,70.0,1229,1249,888,20,6
3,240.0,1295,1377,440,82,2
4,262.0,752,990,1476,238,4


In [0]:
Y_train_ini = answers_df

## Model validation

In [0]:
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()

def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

In [0]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X_train_ini, Y_train_ini, test_size=0.3,random_state=17)

In [0]:
base_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
results = base_model.predict(X_holdout)
result_df = pd.DataFrame(results, columns=['at_least_one','at_least_two','at_least_three'])

In [0]:
score = get_smoothed_mean_log_accuracy_ratio(y_holdout, result_df)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
score

410.59

In [0]:
score_list = []
for i in np.arange(1,20):
    X_train, X_holdout, y_train, y_holdout = train_test_split(X_train_ini, Y_train_ini, test_size=0.3,random_state=i)
    base_model.fit(X_train, y_train)
    results = base_model.predict(X_holdout)
    result_df = pd.DataFrame(results, columns=['at_least_one','at_least_two','at_least_three'])
    score = get_smoothed_mean_log_accuracy_ratio(y_holdout, result_df)
    score_list.append(score)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **

In [0]:
np.mean(score_list)

388.038947368421